# Mental Health Classification with Neural Network

In [1]:
print(2)

2


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import math
import tensorflow as tf
import keras
import keras_tuner as kt

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Activation, Dropout, InputLayer, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from keras import layers, Sequential
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale, OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from keras import backend as K

In [30]:
df = pd.read_csv('../datasets/mentalhealth.csv')
# df = df.iloc[0:3000]
df.head()

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,8/27/2014 11:29,Female,United States,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,8/27/2014 11:31,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,8/27/2014 11:32,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,8/27/2014 11:37,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,8/27/2014 11:43,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes


In [31]:
df.drop(['Timestamp', 'Gender', 'Country'], axis=1, inplace=True)

In [32]:
df.head()

,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes


## Mencari info dari data

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292364 entries, 0 to 292363
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Occupation               292364 non-null  object
 1   self_employed            287162 non-null  object
 2   family_history           292364 non-null  object
 3   treatment                292364 non-null  object
 4   Days_Indoors             292364 non-null  object
 5   Growing_Stress           292364 non-null  object
 6   Changes_Habits           292364 non-null  object
 7   Mental_Health_History    292364 non-null  object
 8   Mood_Swings              292364 non-null  object
 9   Coping_Struggles         292364 non-null  object
 10  Work_Interest            292364 non-null  object
 11  Social_Weakness          292364 non-null  object
 12  mental_health_interview  292364 non-null  object
 13  care_options             292364 non-null  object
dtypes: object(14)
memory

In [7]:
df['self_employed'].dropna(inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292364 entries, 0 to 292363
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Occupation               292364 non-null  object
 1   self_employed            287162 non-null  object
 2   family_history           292364 non-null  object
 3   treatment                292364 non-null  object
 4   Days_Indoors             292364 non-null  object
 5   Growing_Stress           292364 non-null  object
 6   Changes_Habits           292364 non-null  object
 7   Mental_Health_History    292364 non-null  object
 8   Mood_Swings              292364 non-null  object
 9   Coping_Struggles         292364 non-null  object
 10  Work_Interest            292364 non-null  object
 11  Social_Weakness          292364 non-null  object
 12  mental_health_interview  292364 non-null  object
 13  care_options             292364 non-null  object
dtypes: object(14)
memory

In [9]:
df.columns

Index(['Occupation', 'self_employed', 'family_history', 'treatment',
       'Days_Indoors', 'Growing_Stress', 'Changes_Habits',
       'Mental_Health_History', 'Mood_Swings', 'Coping_Struggles',
       'Work_Interest', 'Social_Weakness', 'mental_health_interview',
       'care_options'],
      dtype='object')

In [10]:
df['treatment'].value_counts()

Yes    147606
No     144758
Name: treatment, dtype: int64

In [11]:
df.replace({
    'self_employed': {
        'Yes': 1,
        'No': 0
    },
    'family_history': {
        'Yes': 1,
        'No': 0
    },
    'treatment': {
        'Yes': 1,
        'No': 0
    },
    'Days_Indoors': {
        'Go out Every day': 1,
        '1-14 days': 2,
        '15-30 days': 3,
        '31-60 days': 4,
        'More than 2 months': 5
    },
    'Growing_Stress': {
        'Yes': 3,
        'Maybe': 2,
        'No': 1
    },
    'Changes_Habits': {
        'Yes': 3,
        'Maybe': 2,
        'No': 1
    },
    'Mental_Health_History': {
        'Yes': 3,
        'Maybe': 2,
        'No': 1
    },
    'Mood_Swings': {
        'Medium': 2,
        'Low': 1,
        'High': 3
    },
    'Coping_Struggles': {
        'Yes': 1,
        'No': 0
    },
    'Work_Interest': {
        'Yes': 3,
        'Maybe': 2,
        'No': 1
    },
    'Social_Weakness': {
        'Yes': 3,
        'No': 1,
        'Maybe': 2
    },
    'mental_health_interview': {
        'Yes': 3,
        'No': 1,
        'Maybe': 2
    },
    'care_options': {
        'Not sure': 2,
        'No': 1,
        'Yes': 3,
        'Ye': 3
    }
}, inplace=True)

In [12]:
for column in df.columns:
    print(f"Column Name: {column} - {df[column].unique()}")

Column Name: Occupation - ['Corporate' 'Student' 'Business' 'Housewife' 'Others']
Column Name: self_employed - [nan  0.  1.]
Column Name: family_history - [0 1]
Column Name: treatment - [1 0]
Column Name: Days_Indoors - [2 1 5 3 4]
Column Name: Growing_Stress - [3 1 2]
Column Name: Changes_Habits - [1 3 2]
Column Name: Mental_Health_History - [3 1 2]
Column Name: Mood_Swings - [2 1 3]
Column Name: Coping_Struggles - [0 1]
Column Name: Work_Interest - [1 2 3]
Column Name: Social_Weakness - [3 1 2]
Column Name: mental_health_interview - [1 2 3]
Column Name: care_options - [2 1 3]


In [13]:
df.tail()

,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
292359,Business,1.0,1,1,3,1,2,1,1,1,1,2,2,2
292360,Business,0.0,1,1,3,1,2,1,1,1,1,2,1,3
292361,Business,0.0,1,0,3,1,2,1,1,1,1,2,1,1
292362,Business,0.0,1,1,3,1,2,1,1,1,1,2,1,3
292363,Business,0.0,1,1,3,1,2,1,1,1,1,2,1,3


In [14]:
df = pd.get_dummies(df, columns=['Occupation'])
df.tail()

,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options,Occupation_Business,Occupation_Corporate,Occupation_Housewife,Occupation_Others,Occupation_Student
292359,1.0,1,1,3,1,2,1,1,1,1,2,2,2,1,0,0,0,0
292360,0.0,1,1,3,1,2,1,1,1,1,2,1,3,1,0,0,0,0
292361,0.0,1,0,3,1,2,1,1,1,1,2,1,1,1,0,0,0,0
292362,0.0,1,1,3,1,2,1,1,1,1,2,1,3,1,0,0,0,0
292363,0.0,1,1,3,1,2,1,1,1,1,2,1,3,1,0,0,0,0


In [15]:
X = df.drop(['mental_health_interview'], axis=1)
y = df['mental_health_interview']

## Split the Data

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

X_train = scale(X_train)
X_test = scale(X_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(f"""
X_train shape = {X_train.shape}
y_train shape = {y_train.shape}
X_test shape = {X_test.shape}
y_test shape = {y_test.shape}
""")


X_train shape = (233891, 17)
y_train shape = (233891, 4)
X_test shape = (58473, 17)
y_test shape = (58473, 4)



## Create Model

In [24]:
new_model = Sequential([
    Input((X_train.shape[1],)),
    Dense(512, activation='relu'),
    # Dense(128, activation='relu'),
    # Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='softmax'),
])

new_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

new_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 512)            │         9,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,268 (44.02 KB)

 Trainable params: 11,268 (44.02 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
new_model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=1024)

Epoch 1/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7720 - loss: 0.7165 - val_accuracy: 0.7956 - val_loss: 0.5941
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7940 - loss: 0.5958 - val_accuracy: 0.7956 - val_loss: 0.5930
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7951 - loss: 0.5928 - val_accuracy: 0.7956 - val_loss: 0.5930
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7947 - loss: 0.5922 - val_accuracy: 0.7956 - val_loss: 0.5926
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7949 - loss: 0.5921 - val_accuracy: 0.7956 - val_loss: 0.5922
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7947 - loss: 0.5921 - val_accuracy: 0.7956 - val_loss: 0.5920
Epoch 7/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7949 - loss: 0.5922 - val_accuracy: 0.7956 - val_loss: 0.5921
Epoch 8/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7930 - loss: 0.5950 - val_accuracy: 0.